In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cheer_utils

# pd.options.mode.copy_on_write = True

In [ ]:
df=cheer_utils.LoadIBTrACS()

In [ ]:
# define dicts for filtering circles for case study areas:
a=np.arange(0,370,10)*np.pi/180
rad=3
lo=-76.5
la=34.75
# test track screening circle
# rad= .3
# lo=-77.75
# la=35
nc_circ={'lo': lo,
         'la': la, 
         'rad': rad,
         'cirx': rad*np.cos(a)+lo,
         'ciry': rad*np.sin(a)+la}

rad=3
lo=-94
la=29.5
tx_circ={'lo': lo,
         'la':  la,
         'rad': rad,
         'cirx': rad*np.cos(a)+lo,
         'ciry': rad*np.sin(a)+la}


dlo, dla = 10, 10
ext={ 0: [tx_circ['lo']-dlo, tx_circ['lo']+dlo, tx_circ['la']-dla, tx_circ['la']+dla],
      1: [nc_circ['lo']-dlo, nc_circ['lo']+dlo, nc_circ['la']-dla, nc_circ['la']+dla]}

sz=20

# add distances to targets
df['dist2nc']=np.sqrt(np.square(df['Longitude']-nc_circ['lo']) + np.square(df['Latitude']-nc_circ['la']))
df['dist2tx']=np.sqrt(np.square(df['Longitude']-tx_circ['lo']) + np.square(df['Latitude']-tx_circ['la']))
df

In [ ]:

# plot all tracks
fig = plt.figure(figsize=(16, 10))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines()
ax.set_xlim([-110, 0])
ax.set_ylim([0, 60])


for i,idx in enumerate(np.unique(df.index)): 
    #print(i,idx)
    #ii=df.index==idx
    #print(ii)
    #break
    x=df.loc[df.index==idx].Longitude
    y=df.loc[df.index==idx].Latitude
    #c=df.loc[df.index==idx].Min_pres
    #c=df.loc[df.index==idx].HollandB
    ax.plot(x, y, linewidth=.1, color='k', transform=ccrs.PlateCarree())
    c=df.loc[df.index==idx].Min_pres.values
    inan=np.isnan(c)
    if not np.all(inan):
        cm=ax.scatter(x=x, y=y, c=c, s=sz, cmap=cheer_utils.cmap, transform=ccrs.PlateCarree(), norm=cheer_utils.norm)

    #axx.plot(x.iloc[0], y.iloc[0], marker='*', color='g')
    #axx.plot(x.iloc[-1], y.iloc[-1], marker='*', color='r')
    #cm=ax.scatter(x=x, y=y, c=c, cmap=cheer_utils.cmap, norm=cheer_utils.norm, s=36)

ax.plot(tx_circ['cirx'],tx_circ['ciry'],transform=ccrs.PlateCarree(),color='r')
ax.plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree(),color='r')

ax.plot(tx_circ['lo'],tx_circ['la'],marker='*',transform=ccrs.PlateCarree())
ax.plot(nc_circ['lo'],nc_circ['la'],marker='*',transform=ccrs.PlateCarree())

cb1 = plt.colorbar(cm, ax=ax, orientation='vertical', pad=0.02, aspect=15, shrink=0.80)
cb1.ax.set_ylabel('[mb]', size=12)
cb1.ax.tick_params(labelsize='large')


In [ ]:
# fig, ax = plt.subplots(nrows=1,ncols=2,
#                            subplot_kw={'projection': ccrs.PlateCarree()},
#                            figsize=(16,7))
# ax=ax.flatten()
    
# ax[0].stock_img()
# ax[0].coastlines()
# ax[1].stock_img()
# ax[1].coastlines()
    
# ax[0].plot(tx_circ['cirx'],tx_circ['ciry'],transform=ccrs.PlateCarree())
# ax[1].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree())

# ax[0].plot(tx_circ['lo'],tx_circ['la'],marker='*',transform=ccrs.PlateCarree())
# ax[1].plot(nc_circ['lo'],nc_circ['la'],marker='*',transform=ccrs.PlateCarree())

# for i in range(0,2):
#     ax[i].axis('equal')
#     ax[i].set_ylim(ext[i][2:4])
#     ax[i].set_xlim(ext[i][0:2])
#     ax[i].grid(True)
#     ax[i].stock_img()
#     ax[i].coastlines()

In [ ]:
# Find the storm index numbers (abssn) that pass thru the screening circles
# This gets the parts of tracks that enter the circles.  The complete track will be extracted later

df_nc=df.loc[(df['dist2nc'] <= nc_circ['rad']) & (df['NATURE']=='TS')]

df_tx=df.loc[(df['dist2tx'] <= tx_circ['rad']) & (df['NATURE']=='TS')]

# & (df['Min_pres'] <= 980)]

# df_nc=df.loc[(df['dist2nc'] <=nc_circ['rad']) & (df['Category'] > 2)]
# df_tx=df.loc[(df['dist2tx'] <=tx_circ['rad']) & (df['Category'] > 2)]
#idx_nc=np.unique(df_nc.index).astype(int)

idx_nc = df_nc.index.unique()
idx_tx = df_tx.index.unique()
#idx_tx=np.unique(df_tx.index).astype(int)
#print(idx_nc)

print(f'Initial number of NC events = {len(idx_nc)}')
print(f'Initial number of TX events = {len(idx_tx)}')
#df_nc['NATURE'].unique()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2,
                       subplot_kw={'projection': ccrs.PlateCarree()},
                       figsize=(16,7))
ax=ax.flatten()
    
for i in range(0,2):
    ax[i].axis('equal')
    ax[i].set_ylim(ext[i][2:4])
    ax[i].set_xlim(ext[i][0:2])
    ax[i].grid(True)
    ax[i].stock_img()
    ax[i].coastlines()
# ax[1].stock_img()
# ax[1].coastlines()
    

for i,idx in enumerate(idx_tx): 
        x=df.loc[df.index==idx].Longitude
        y=df.loc[df.index==idx].Latitude
        #c=df.loc[df.index==idx].HollandB
        ax[0].plot(x, y, linewidth=1, color='k') # ,marker='o')
        
        c=df.loc[df.index==idx].Min_pres.values
        if not np.all(np.isnan(c)):
            cm=ax[0].scatter(x=x, y=y, c=c, s=sz, cmap=cheer_utils.cmap) # , norm=cheer_utils.norm)
        
ax[0].plot(tx_circ['cirx'],tx_circ['ciry'],transform=ccrs.PlateCarree(),color='r')

for i,idx in enumerate(idx_nc): 
        #if i==0:
        #print(i,idx)
        x=df.loc[df.index==idx].Longitude
        y=df.loc[df.index==idx].Latitude
        #c=df.loc[df.index==idx].HollandB
        ax[1].plot(x, y, linewidth=1, color='k') # ,marker='o')

        c=df.loc[df.index==idx].Min_pres.values
        if not np.all(np.isnan(c)):
            cm=ax[1].scatter(x=x, y=y, c=c, s=sz, cmap=cheer_utils.cmap) # , norm=cheer_utils.norm)
        #break
            
ax[1].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree(),color='r')

# cb1 = plt.colorbar(cm, ax=ax[1], orientation='vertical', pad=0.02, aspect=15) # , shrink=0.15)
# cb1.ax.set_ylabel('[mb]', size=12)
# cb1.ax.tick_params(labelsize='large')
        

In [ ]:
print(i,idx)
fig, ax = plt.subplots(nrows=1,ncols=1,
                       #subplot_kw={'projection': ccrs.PlateCarree()},
                       figsize=(16,7))
plt.plot(nc_circ['cirx'],nc_circ['ciry'],color='r')
x=df.loc[df.index==idx].Longitude
y=df.loc[df.index==idx].Latitude
ax.plot(x,y)
ax.axis('equal')
# ax.set_ylim(ext[1][2:4])
# ax.set_xlim(ext[1][0:2])
ax.grid(True)
x=df_nc.loc[df_nc.index==idx].Longitude
y=df_nc.loc[df_nc.index==idx].Latitude
ax.plot(x,y,color='y')


In [ ]:
df_nc.loc[df_nc.index==idx]

In [ ]:
axx=(-(df_nc['Min_pres']-1020)).hist( density=True, bins=range(0,120,10))
axx.set_ylim([0, .040])
axx.set_xlabel('Delta-P')

In [ ]:
df_nc

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2,
                       subplot_kw={'projection': ccrs.PlateCarree()},
                       figsize=(16,7))
ax=ax.flatten()
    
for i in range(0,2):
    ax[i].axis('equal')
    ax[i].set_ylim(ext[i][2:4])
    ax[i].set_xlim(ext[i][0:2])
    ax[i].grid(True)
    ax[i].stock_img()
    ax[i].coastlines()
    
ax[0].plot(tx_circ['cirx'],tx_circ['ciry'],transform=ccrs.PlateCarree(),color='r')
ax[1].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree(),color='r')
df_tx.plot(x='Longitude',y='Latitude',ax=ax[0],linestyle='',marker='.',transform=ccrs.PlateCarree())
df_nc.plot(x='Longitude',y='Latitude',ax=ax[1],linestyle='',marker='.',transform=ccrs.PlateCarree())
